In [1]:
import shutil
import pyterrier as pt
pt.init()

PyTerrier 0.3.1 has loaded Terrier 5.4 (built by craigm on 2021-01-16 14:17)


In [2]:
checkpoint="/users/tr.craigm/projects/pyterrier/pyterrier_bert/ColBERT/GOOD_MODELS/colbert.dnn"
index_root="/tmp/vaswani_colbert"
index_name="tesdting"

dataset = pt.get_dataset("irds:vaswani")

In [3]:
import pyterrier_colbert.indexing
from importlib import reload
reload(pyterrier_colbert.indexing)

<module 'pyterrier_colbert.indexing' from '/users/tr.craigm/projects/pyterrier/pyterrier_colbert/pyterrier_colbert/indexing.py'>

In [4]:
#TODO what is chunksize
indexer = pyterrier_colbert.indexing.ColBERTIndexer(checkpoint, index_root, index_name, chunksize=3)
if os.path.exists(indexer.args.index_path):
    shutil.rmtree(indexer.args.index_path)

In [5]:
indexer.index(dataset.get_corpus_iter(), num_docs=11_429)

[Feb 21, 14:26:10] [0] 		 #> Local args.bsize = 128
[Feb 21, 14:26:10] [0] 		 #> args.index_root = /tmp/vaswani_colbert
[Feb 21, 14:26:10] [0] 		 #> self.possible_subset_sizes = [69905]
Some weights of the model checkpoint at bert-base-uncased were not used when initializing ColBERT: ['cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.decoder.weight', 'cls.seq_relationship.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias']
- This IS expected if you are initializing ColBERT from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPretraining model).
- This IS NOT expected if you are initializing ColBERT from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequen

In [6]:
pytcolbert = indexer.ranking_factory(memtype="mem")

#this is equivalent
#pytcolbert = pyterrier_colbert.ranking.ColBERTFactory(checkpoint, index_root, index_name, memtype="mem")



In [7]:
dataset = pt.get_dataset("vaswani")
bm25 = pt.BatchRetrieve(dataset.get_index(), wmodel="BM25")
colbert_e2e = pytcolbert.end_to_end()
pt.Experiment(
    [bm25, colbert_e2e],
    dataset.get_topics(),
    dataset.get_qrels(),
    eval_metrics=["map"],
    baseline=0,
    names = ["BM25", "ColBERT E2E"]
)

 14:27:53] #> Searching from 0 to 32...
[Feb 21, 14:27:53] #> Lookup the PIDs..
[Feb 21, 14:27:53] #> Converting to a list [shape = torch.Size([1, 32000])]..
[Feb 21, 14:27:53] #> Removing duplicates (in parallel if large enough)..
[Feb 21, 14:27:53] #> Done with embedding_ids_to_pids().
qid 64 retrieved docs 5864
64q [00:23,  3.19q/s][Feb 21, 14:27:54] #> Search in batches with faiss. 		 Q.size() = torch.Size([1, 32, 128]), Q_faiss.size() = torch.Size([32, 128])
[Feb 21, 14:27:54] #> Searching from 0 to 32...
[Feb 21, 14:27:54] #> Lookup the PIDs..
[Feb 21, 14:27:54] #> Converting to a list [shape = torch.Size([1, 32000])]..
[Feb 21, 14:27:54] #> Removing duplicates (in parallel if large enough)..
[Feb 21, 14:27:54] #> Done with embedding_ids_to_pids().
qid 65 retrieved docs 5053
65q [00:23,  3.23q/s][Feb 21, 14:27:54] #> Search in batches with faiss. 		 Q.size() = torch.Size([1, 32, 128]), Q_faiss.size() = torch.Size([32, 128])
[Feb 21, 14:27:54] #> Searching from 0 to 32...
[Feb 21,

,name,map,map +,map -,map p-value
0,BM25,0.296517,NaN,NaN,NaN
1,ColBERT E2E,0.278845,49.0,44.0,0.231719
